<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dataset-description:" data-toc-modified-id="Dataset-description:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset description:</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Cleaning</a></span><ul class="toc-item"><li><span><a href="#Informational" data-toc-modified-id="Informational-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Informational</a></span></li><li><span><a href="#Exit-Rates" data-toc-modified-id="Exit-Rates-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Exit Rates</a></span></li><li><span><a href="#Browser" data-toc-modified-id="Browser-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Browser</a></span></li><li><span><a href="#Revenue" data-toc-modified-id="Revenue-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Revenue</a></span></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data/online_shoppers_intention_DATAPTDIC19.csv', sep=';')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

## Dataset description:
 - Administrative:`float`. Administrative Value. `yet to identify`.
 - Administrative_Duration: `object`. Duration in Administrative Page. `Identify values and change dtype accordingly.`. 
 - Informational: `float`. Informational Value. `yet to identify` 
 - Informational_Duration: `object`. Duration in Informational Page. `Identify values and change dtype accordingly.`
 - ProductRelated: `float.` Product Related Value. `yet to identify` 
 - ProductRelated_Duration: `object`. Duration in Product Related Page. `Identify values and change dtype accordingly.` 
 - BounceRates: `float`. Bounce Rates of a web page. Percentages. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. 
 - ExitRates: `float`. Exit rate of a web page. The value of "Exit Rate" feature for a specific web page is calculated as for all pageviews to the page, the percentage that were the last in the session. 
 - PageValues: `object`. Page values of each web page. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction. `Identify values and change dtype accordingly.` 
 - SpecialDay: Special days like valentine etc. `float`. Closeness to a special date. `dtype correct`. For example, for Valentina’s day, this value takes a nonzero value between February 2 and February 12, zero before and after this date unless it is close to another special day, and its maximum value of 1 on February 8. 
 - Month: Month of the year. `object`. String to identify the month of the year. `clean`. 
 - OperatingSystems: Operating system used. `float`. `Try to explain the values`. 
 - Browser: Browser used. `integer` 
 - Region: Region of the user. `integer`. 
 - TrafficType: Traffic Type. `integer`.  
 - VisitorType: Types of Visitor `object` 
 - Weekend: Weekend or not `bool`  
 - Revenue: Revenue will be generated or not `object`. `Should be bool` 
     

## Data Cleaning

### Informational

In [ ]:
print(f'Total number of unique values: {data.Informational.nunique()}')
print(f'Total number of missing values: {data.Informational.isnull().sum()}')

# Filterring to check where have missing values

null_columns = data.columns[data.isnull().any()]
null_data = data[data["Informational"].isnull()][null_columns]
null_data

Notes:
- **Data Type**: Numeric variable, float. `Correct`
- **Errors**: Have 17 unique categories (pages). `Don't know what 0 stands for in this case, if it's a category of a page, I would suggest replacing it with another number to avoid Null warnings`
- **Missing values (NaN)**: Have 14 missing values (NaN), which is 0.1% out of all dataset. In all the 14 cases of NaN values, the data is also missing in almost all of other variables. `Will change it to average only can also do the same in other columns`
- **Outliers**: no outliers

### Exit Rates

Notes:
- **Data Type**: Numeric variable, float. `Correct`
- `Exit Rates is highly correlated with BounceRates and should be ignored for analysis`

### Browser

In [ ]:
print(f'Total number of unique values: {data.Browser.nunique()}')
print(f'Total number of missing values: {data.Browser.isnull().sum()}')

Notes:
- **Data Type**: Numeric integer. `Correct`
- **Errors**: Have 13 unique categories (browsers), no errors
- **Missing values (NaN)**: no missing values 
- **Outliers**: no outliers

### Revenue

In [ ]:
print(f'Total number of missing values: {data.Revenue.isnull().sum()}')
print(f'Total number of unique values: {data.Revenue.nunique()}')
print(data.Revenue.value_counts())

Notes:
- **Data Type**: Object. `Will convert to boolean`
- **Errors**: Have 4 categories instead of 2. `Will unify all to TRUE and FALSE`
- **Missing values (NaN)**: no missing values
- **Outliers**: no outliers